### Imports

In [33]:
import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
from tensorflow import keras
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from datetime import datetime
from copy import deepcopy

### Loading and Processing Data

## Stock Price Forecasting

### Preparing Data

In [ ]:
sp500 = sp500_og.copy()[["Close"]]
sp500.index = sp500.index.map(lambda ts: ts.date())
sp500.tail()

,Close
Date,
2023-07-17,4522.790039
2023-07-18,4554.979980
2023-07-19,4565.720215
2023-07-20,4534.870117
2023-07-21,4536.339844


In [ ]:
def window_data(data, n=3):
    windowed_data = pd.DataFrame()
    for i in range(n, 0, -1):
        windowed_data[f'Target-{i}'] = data['Close'].shift(i)
    windowed_data['Target'] = data['Close']
    return windowed_data.dropna()

In [ ]:
windowed = window_data(sp500) # gets us the values before the target, and then the target that we actually predict
windowed.head()

,Target-3,Target-2,Target-1,Target
Date,,,,
1990-01-05,359.690002,358.760010,355.670013,352.200012
1990-01-08,358.760010,355.670013,352.200012,353.790009
1990-01-09,355.670013,352.200012,353.790009,349.619995
1990-01-10,352.200012,353.790009,349.619995,347.309998
1990-01-11,353.790009,349.619995,347.309998,348.529999


In [ ]:
def df_to_date_X_y(df):
    dates = df.index
    dates = pd.to_datetime(dates)
    df = df.to_numpy()
    middle_matrix = df[:, :-1]
    X = middle_matrix.reshape((len(dates), middle_matrix.shape[1], 1))
    X = X.astype(np.float32)
    y = df[:, -1].astype(np.float32)
    return (dates, X, y)

In [ ]:
(dates, X, y) = df_to_date_X_y(windowed)
dates.shape, X.shape, y.shape

((8450,), (8450, 3, 1), (8450,))

In [ ]:
q_80 = int(len(dates)*.8)
dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]
dates_test, X_test, y_test = dates[q_80:], X[q_80:], y[q_80:]

### Creating and Training First Model

In [ ]:
model = keras.Sequential([
        keras.layers.Input((3,1)),
        keras.layers.LSTM(units=64, return_sequences=True), # LSTM model is good for predicting series (i.e forecasting stocks
        keras.layers.LSTM(units=32),
        keras.layers.Dense(units=128, activation="relu"),
        keras.layers.Dense(units=128, activation="relu"),
        keras.layers.Dense(units=64, activation="relu"),
        keras.layers.Dense(units=1, activation="linear")
    ])

In [ ]:
def compile_fit_model(model, X_train, y_train):
    model.compile(optimizer=keras.optimizers.legacy.Adam(learning_rate=0.001),
              loss=keras.losses.MeanSquaredError(),
              metrics=["mean_absolute_error"])

    model.fit(x=X_train, y=y_train, epochs=112)

In [ ]:
compile_fit_model(model, X_train, y_train)

Epoch 1/112


212/212 [==============================] - 2s 2ms/step - loss: 4067.0042 - mean_absolute_error: 31.2583
Epoch 2/112
212/212 [==============================] - 0s 2ms/step - loss: 311.5945 - mean_absolute_error: 12.6936
Epoch 3/112
212/212 [==============================] - 0s 2ms/step - loss: 395.6688 - mean_absolute_error: 14.4587
Epoch 4/112
212/212 [==============================] - 0s 2ms/step - loss: 479.7789 - mean_absolute_error: 16.3584
Epoch 5/112
212/212 [==============================] - 0s 2ms/step - loss: 724.3039 - mean_absolute_error: 20.2156
Epoch 6/112
212/212 [==============================] - 0s 2ms/step - loss: 596.4030 - mean_absolute_error: 17.7206
Epoch 7/112
212/212 [==============================] - 0s 2ms/step - loss: 475.6990 - mean_absolute_error: 16.0363
Epoch 8/112
212/212 [==============================] - 0s 2ms/step - loss: 1248.8832 - mean_absolute_error: 23.5722
Epoch 9/112
212/212 [==============================] - 0s 2ms/step - loss: 583.9874 - mean

### Plotting Observations

In [ ]:
def plot_train(X_train, y_train):
    train_predictions = model.predict(X_train).flatten()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates_train, y=train_predictions, mode="lines", name="Training Predictions"))
    fig.add_trace(go.Scatter(x=dates_train, y=y_train, mode="lines", name="Training Observations"))
    fig.update_layout(xaxis_title="Dates", yaxis_title="Value")
    fig.show()

In [ ]:
plot_train(X_train, y_train)

212/212 [==============================] - 0s 755us/step


In [ ]:
def plot_test(X_test, y_test):
    test_predictions = model.predict(X_test).flatten()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates_test, y=test_predictions, mode="lines", name="Testing Predictions"))
    fig.add_trace(go.Scatter(x=dates_test, y=y_test, mode="lines", name="Testing Observations"))
    fig.update_layout(xaxis_title="Dates", yaxis_title="Value")
    fig.show()

In [ ]:
plot_test(X_test, y_test)

 1/53 [..............................] - ETA: 0s

53/53 [==============================] - 0s 835us/step


#### Change Model to Predict Recursively for 3 Years

In [ ]:
start_date = pd.Timestamp("2020-07-23").date()
windowed_yr = windowed.loc[windowed.index >= start_date]
windowed_yr.head()

,Target-3,Target-2,Target-1,Target
Date,,,,
2020-07-23,3251.840088,3257.300049,3276.020020,3235.659912
2020-07-24,3257.300049,3276.020020,3235.659912,3215.629883
2020-07-27,3276.020020,3235.659912,3215.629883,3239.409912
2020-07-28,3235.659912,3215.629883,3239.409912,3218.439941
2020-07-29,3215.629883,3239.409912,3218.439941,3258.439941


In [ ]:
(dates, X, y) = df_to_date_X_y(windowed_yr)
dates.shape, X.shape, y.shape

((754,), (754, 3, 1), (754,))

In [ ]:
q_80 = int(len(dates)*.8)
dates_train, X_train, y_train = dates[:q_80], X[:q_80], y[:q_80]
dates_test, X_test, y_test = dates[q_80:], X[q_80:], y[q_80:]

In [ ]:
compile_fit_model(model, X_train, y_train)

Epoch 1/112
19/19 [==============================] - 1s 3ms/step - loss: 216442.8281 - mean_absolute_error: 362.5095
Epoch 2/112
19/19 [==============================] - 0s 2ms/step - loss: 49603.1719 - mean_absolute_error: 185.6429
Epoch 3/112
19/19 [==============================] - 0s 2ms/step - loss: 28725.3770 - mean_absolute_error: 138.6058
Epoch 4/112
19/19 [==============================] - 0s 2ms/step - loss: 24820.1016 - mean_absolute_error: 131.4650
Epoch 5/112
19/19 [==============================] - 0s 2ms/step - loss: 15801.3877 - mean_absolute_error: 101.4601
Epoch 6/112
19/19 [==============================] - 0s 2ms/step - loss: 9700.5215 - mean_absolute_error: 75.0098
Epoch 7/112
19/19 [==============================] - 0s 2ms/step - loss: 7176.4072 - mean_absolute_error: 64.9727
Epoch 8/112
19/19 [==============================] - 0s 2ms/step - loss: 5911.8281 - mean_absolute_error: 60.9443
Epoch 9/112
19/19 [==============================] - 0s 2ms/step - loss: 4940

In [ ]:
plot_train(X_train, y_train)

19/19 [==============================] - 0s 920us/step


In [ ]:
plot_test(X_test, y_test)

5/5 [==============================] - 0s 2ms/step


### Long-Term Recursive Prediction

In [ ]:
recursive_predictions = []
for target_date in dates_test:
    last_window = deepcopy(X_train[-1])
    next_pred = model.predict(np.array([last_window])).flatten()
    recursive_predictions.append(next_pred)
    last_window[-1] = next_pred

1/1 [==============================] - 0s 10ms/step


In [ ]:
def plot_recursive(y_test, recursive_predictions):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates_test, y=recursive_predictions, mode="lines", name="Testing Predictions"))
    fig.add_trace(go.Scatter(x=dates_test, y=y_test, mode="lines", name="Testing Observations"))
    fig.update_layout(xaxis_title="Dates", yaxis_title="Value")
    fig.show()

In [ ]:
plot_recursive(y_test, recursive_predictions)

In [ ]:
# yea, that didnt't work... looks like it's hard to predict stocks
# let's try atleast predicting if the stock will go up or down

In [ ]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)
predictions_new = backtest(sp500, model, new_predictors)

In [ ]:
predictions_new["Target"].value_counts

<bound method IndexOpsMixin.value_counts of Date
2003-11-14 00:00:00-05:00    0
2003-11-17 00:00:00-05:00    0
2003-11-18 00:00:00-05:00    1
2003-11-19 00:00:00-05:00    0
2003-11-20 00:00:00-05:00    1
                            ..
2023-07-14 00:00:00-04:00    1
2023-07-17 00:00:00-04:00    1
2023-07-18 00:00:00-04:00    1
2023-07-19 00:00:00-04:00    0
2023-07-20 00:00:00-04:00    1
Name: Target, Length: 4952, dtype: int64>

In [ ]:
precision_score(predictions_new["Target"], predictions_new["Predictions"])

0.5691554467564259

In [2]:
# this model will be trained on historical data of the S&P 500 Index as a general indicator of the market
# NOTE: data is up to 2020, this is so model is more generalizable without data from COVID years
sp500_og = yf.Ticker("^GSPC")
sp500_og = sp500_og.history(period="max")

In [3]:
sp500_og = sp500_og.drop(["Dividends", "Stock Splits"], axis=1)
sp500_og = sp500_og.loc["1990-01-01":].copy() # only use data after 1990
sp500_og.head()

,Open,High,Low,Close,Volume
Date,,,,,
1990-01-02 00:00:00-05:00,353.399994,359.690002,351.980011,359.690002,162070000
1990-01-03 00:00:00-05:00,359.690002,360.589996,357.890015,358.760010,192330000
1990-01-04 00:00:00-05:00,358.760010,358.760010,352.890015,355.670013,177000000
1990-01-05 00:00:00-05:00,355.670013,355.670013,351.350006,352.200012,158530000
1990-01-08 00:00:00-05:00,352.200012,354.239990,350.540009,353.790009,140110000


In [4]:
px.line(sp500_og, y="Close")

## Stock Price Movement Prediction 

### Preparing Data

In [5]:
sp500 = sp500_og.copy()
sp500["Tomorrow"] = sp500["Close"].shift(-1)
sp500["Target"] = (sp500["Tomorrow"] > sp500["Close"]).astype(int) # we want to predict if price goes up or down tommorow

In [6]:
# X_train, X_test, y_train, y_test = train_test_split(x=pd.get_dummies(sp500), y=sp500["Target"], test_size=.2)

### Creating and Testing First Model

In [7]:
model = RandomForestClassifier(n_estimators=100, min_samples_split=100, random_state=1) # by locking random state we can more easily control our model
# NOTE: this data will be re-split later, this is just a start
train = sp500.iloc[:100] # all but last 100 rows
test = sp500.iloc[-100:] # only last 100 rows
predictors = ["Close", "Volume", "Open", "High", "Low"]

In [8]:
model.fit(X=train[predictors], y=train["Target"])
preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index, name="Predictions")
precision_score(test["Target"], preds)
combined = pd.concat([test["Target"], preds], axis=1)

In [9]:
px.line(combined)

### Backtesting Model

In [10]:
def predict(train, test, predictors, model):
    model.fit(X=train[predictors], y=train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >= .6] = 1
    preds[preds < .6] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    precision_score(test["Target"], preds)
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [11]:
pred1 = predict(train, test, predictors, model)
px.line(pred1)

/Users/anwar/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



In [12]:
def backtest(data, model, predictors, start=2500, step=250): # start with 10 years and predict one year at a time
    all_preds = []
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        preds = predict(train, test, predictors, model)
        all_preds.append(preds)
    return pd.concat(all_preds)

In [13]:
predictions = backtest(sp500, model, predictors)

/Users/anwar/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/anwar/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/anwar/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/anwar/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to cont

In [14]:
predictions["Predictions"].value_counts()

Predictions
0.0    5044
1.0     909
Name: count, dtype: int64

In [15]:
precision_score(predictions["Target"], predictions["Predictions"]) # model is not very good right now :(

0.5324532453245324

#### Adding Predictors

In [16]:
horizons = [2,5,60,250,1000] # look at means at each of these horizons
new_predictors = []
for h in horizons:
    rolling_avgs = sp500.rolling(h).mean()
    
    ratio_col = f"Close_Ratio_{h}"
    sp500[ratio_col] = sp500["Close"] / rolling_avgs["Close"]
    trend_col = f"Trend_{h}"
    sp500[trend_col] = sp500.shift(1).rolling(h).sum()["Target"]

    new_predictors += [ratio_col, trend_col]

In [17]:
sp500 = sp500.dropna()
sp500.tail()

,Open,High,Low,Close,Volume,Tomorrow,Target,Close_Ratio_2,Trend_2,Close_Ratio_5,Trend_5,Close_Ratio_60,Trend_60,Close_Ratio_250,Trend_250,Close_Ratio_1000,Trend_1000
Date,,,,,,,,,,,,,,,,,
2023-07-14 00:00:00-04:00,4514.609863,4527.759766,4499.560059,4505.419922,3647450000,4522.790039,1,0.999488,1.0,1.008537,4.0,1.058002,32.0,1.117622,120.0,1.184298,532.0
2023-07-17 00:00:00-04:00,4508.859863,4532.850098,4504.899902,4522.790039,3538240000,4554.979980,1,1.001924,1.0,1.007318,4.0,1.060553,33.0,1.121161,121.0,1.188390,533.0
2023-07-18 00:00:00-04:00,4521.779785,4562.299805,4514.589844,4554.979980,4090010000,4565.720215,1,1.003546,2.0,1.009285,4.0,1.066329,34.0,1.128448,121.0,1.196367,533.0
2023-07-19 00:00:00-04:00,4563.870117,4578.430176,4557.479980,4565.720215,4115670000,4534.870117,0,1.001178,2.0,1.007487,4.0,1.067044,34.0,1.130431,121.0,1.198702,534.0
2023-07-20 00:00:00-04:00,4554.379883,4564.740234,4527.560059,4534.870117,3761770000,4536.339844,1,0.996610,1.0,0.999584,3.0,1.058194,33.0,1.122197,120.0,1.190127,534.0


### Updating Model and Re-Predicting

In [ ]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)
predictions_new = backtest(sp500, model, new_predictors)

In [ ]:
predictions_new["Target"].value_counts

In [ ]:
precision_score(predictions_new["Target"], predictions_new["Predictions"])

In [ ]:
# this model is pretty decent at predicting the movements of the stock market